In [ ]:
!python main.py

In [ ]:
# Master reset all databases and posts
# Need quite a few SERP API credits to run this, do not try this too often

from main import reset

reset()

In [ ]:
# Only remake all .md posts
from scholar.post_maker import make_all

make_all(overwrite=True)

# Archive

Contains 1 off snippets to make change to the system.

## Add new column called arxiv_term in SQLite

In [ ]:
# import sqlite3

# Drop column
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_group_tag;')
#     conn.commit()

# with sqlite3.connect("scholar/paper.db") as conn:
#     c = conn.cursor()
#     c.execute("ALTER TABLE paper ADD COLUMN category VARCHAR(255);")
#     conn.commit()

## Compute new column

In [ ]:
# import sqlite3
# import pandas as pd
# from scholar.database import Paper

# with sqlite3.connect("scholar/paper.db") as conn:

#     # Add journal column
#     cursor = conn.cursor()
#     cursor.execute("ALTER TABLE paper ADD COLUMN journal VARCHAR(255);")

#     # Export data to pandas for easier manipulation
#     df = pd.read_sql_query("SELECT * FROM paper", conn)


# # Extract journal name from publication_info_summary
# journals = df.publication_info_summary.str.split(" - ", expand=True)[2]

# # Update the database
# for record, new_value in zip(Paper.select(), journals):
#     record.journal = new_value
#     record.save()

## Patch arxiv

In [ ]:
from scholar.database import Paper
from scholar.api import query_arxiv


papers = Paper.select().where(
    Paper.journal == "arxiv.org" and Paper.arxiv_category_tag.startswith("astro-ph")
)

for paper in papers:
    print(paper.arxiv_id, paper.arxiv_category_tag, paper.category)
    # paper.category = "Astrophysics"
    # paper.save()

In [ ]:
from scholar.api import get_bibtex

In [ ]:
get_bibtex("2304.06806")